In [16]:
import langchain_community
from langchain.chains import create_sql_query_chain
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
import pyodbc
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.callbacks import get_openai_callback
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
import os

In [17]:
# Replace these values with your actual SQL Server credentials
server = 'LOCALHOST'
database = 'REALESTATE'

In [18]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#"sk-proj-HlzQnTbGrHupQ5IIBTyyT3BlbkFJi2jOsF0cluhi8vseaMht"
# Create the connection string
connection_string = f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

In [19]:
db = SQLDatabase.from_uri(connection_string)

In [20]:
toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY))
context = toolkit.get_context()
tools = toolkit.get_tools()

In [21]:
langchain_community.agent_toolkits.sql.prompt.SQL_SUFFIX

'Begin!\n\nQuestion: {input}\nThought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.\n{agent_scratchpad}'

In [22]:
SQL_SUFFIX1=langchain_community.agent_toolkits.sql.prompt.SQL_SUFFIX= '''Begin!

Question: {input}
Previous chat history: {chat_history}
Thought: If the question is general (e.g., introductions, greetings, or other non-database-related topics), respond using human interaction without querying the database.

{agent_scratchpad}
'''


In [23]:
SQL_FUNCTIONS_SUFFIX1 = """
I need to perform the following steps to answer the user's query effectively and only use the SQL Server database system:
I MUST NOT use the tool if I found the answer in the bervious chat {chat_history} just call it
1. **Check if the Question is General**:
    - Determine if the user's question is a general conversational input or a database-related query.
    - If the question is general (e.g., introductions, greetings, or other non-database-related topics), respond conversationally without querying the database.
          Example of general questions and responses:
                - input: HumanMessage(content='my name is Samira')
                output: AIMessage(content='Nice to meet you, Samira. How can I assist you today?')
                - input: HumanMessage(content='How are you?')
                output: AIMessage(content='I am an AI assistant, so I don't have feelings, but I'm here to help you!')

2. **Paraphrase the Question for Clarity**:
    - Rephrase the user's question to ensure clarity and accurately capture the intent.

3. **Identify Relevant Tables**:
    - Look at the tables in the database to identify which tables are relevant to the question.

4. **Query the Schema**:
    - Examine the schema of the relevant tables to understand their structure and relationships.

5. **Determine the Need for Joins**:
    - Assess whether the question requires combining data from multiple tables.
    - If joins are necessary, identify the keys and relationships to use in the join.

6. **Construct the SQL Query**:
    - Write the SQL query using SQL Server syntax.
    - Ensure the query includes joins if needed to gather data from multiple tables.

7. **Format the Results**:
    - Present the query results in a clear and understandable format for the user.
    - After constructing the query, execute it on the SQL Server and present the output as the ANSWER.

8. **Respond Appropriately**:
    - If the user asks a general question, respond with a general answer (DON'T use the tool and DON'T make a query).
    - If the user asks a database-related question, follow the steps above to query the database and provide an accurate response.

Proceeding with these steps will ensure that I accurately understand and process the user's query, including handling joins and using the appropriate SQL Server syntax.
"""

In [24]:
SQL_FUNCTIONS_SUFFIX1 = """
I MUST NOT use the tool if I found the answer in the bervious chat {chat_history} just call it
 - If the question is general (e.g., introductions, greetings, or other non-database-related topics), respond conversationally without querying the database.
          Example of general questions and responses:
                - input: HumanMessage(content='my name is Samira')
                output: AIMessage(content='Nice to meet you, Samira. How can I assist you today?')
                - input: HumanMessage(content='How are you?')
                output: AIMessage(content='I am an AI assistant, so I don't have feelings, but I'm here to help you!')

I accurately understand and process the user's query, including handling joins and using the appropriate SQL Server syntax.
"""


In [25]:
messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="chat_history"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX1),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)

In [26]:
 """
I need to perform the following steps to answer the user's query effectively and only use sql server database system.:

1. **Paraphrase the Question for Clarity**:
    - Rephrase the user's question to ensure clarity and capture the intent accurately.
    - if ask general question you respone general answer for example hi or how are you.

2. **Identify Relevant Tables**:
    - Look at the tables in the database to identify which tables are relevant to the question.

3. **Query the Schema**:
    - Examine the schema of the relevant tables to understand their structure and relationships.

4. **Determine the Need for Joins**:
    - Assess whether the question requires combining data from multiple tables.
    - If joins are necessary, identify the keys and relationships to use in the join.

5. **Construct the SQL Query**:
    - Write the SQL query using SQL Server syntax.
    - Ensure the query includes joins if needed to gather data from multiple tables.

7. **Format the Results**:
    - Present the query results in a clear and understandable format for the user.after i got the query I should go to sql server to proceed the query the give the output as ANSWER.

Proceeding with these steps will ensure that I accurately understand and process the user's query, including handling joins and using the appropriate SQL Server syntax.
"""

"\nI need to perform the following steps to answer the user's query effectively and only use sql server database system.:\n\n1. **Paraphrase the Question for Clarity**:\n   - Rephrase the user's question to ensure clarity and capture the intent accurately.\n   - if ask general question you respone general answer for example hi or how are you.\n\n2. **Identify Relevant Tables**:\n   - Look at the tables in the database to identify which tables are relevant to the question.\n\n3. **Query the Schema**:\n   - Examine the schema of the relevant tables to understand their structure and relationships.\n\n4. **Determine the Need for Joins**:\n   - Assess whether the question requires combining data from multiple tables.\n   - If joins are necessary, identify the keys and relationships to use in the join.\n\n5. **Construct the SQL Query**:\n   - Write the SQL query using SQL Server syntax.\n   - Ensure the query includes joins if needed to gather data from multiple tables.\n\n7. **Format the Re

In [27]:
MessagesPlaceholder

langchain_core.prompts.chat.MessagesPlaceholder

In [28]:
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='chat_history'),
 AIMessage(content="\nI MUST NOT use the tool if I found the answer in the bervious chat {chat_history} just call it\n - If the question is general (e.g., introductions, greetings, or other non-database-related topics), respond conversationally without querying the database.\n          Example of general questions and responses:\n                - input: HumanMessage(content='my name is Samira')\n                output: AIMessage(content='Nice to meet you, Samira. How can I assist you today?')\n                - input: HumanMessage(content='How are you?')\n                output: AIMessage(content='I am an AI assistant, so I don't have feelings, but I'm here to help you!')\n\nI accurately understand and process the user's query, including handling joins and using the appropriate SQL Server syntax.\n"),
 MessagesPlaceholder(variable_name=

In [29]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

In [30]:
memory.load_memory_variables({})

{'chat_history': []}

In [31]:
from langchain.agents import create_openai_tools_agent , create_sql_agent
from langchain.agents.agent import AgentExecutor
from langchain.chains import ConversationChain


llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=OPENAI_API_KEY)

agent = create_openai_tools_agent(llm, tools, prompt)
#gpt-3.5-turbo-0613

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,memory=memory)
agent_executor.invoke({"input": "what is my name?"})
#what is the tables do you have?
#I WANT THE HIGHEST MONTHLY SALARY?
#Calculates the total monthly salary and total current debts for each buyer.
#Total Rental Income and Operating Expenses by Location
#Properties with High Return on Investment



> Entering new AgentExecutor chain...
Nice to meet you! How can I assist you today?

> Finished chain.


{'input': 'what is my name?',
 'chat_history': [HumanMessage(content='what is my name?'),
  AIMessage(content='Nice to meet you! How can I assist you today?')],
 'output': 'Nice to meet you! How can I assist you today?'}

In [32]:
agent.agent.llm_chain.prompt.template

AttributeError: 'RunnableSequence' object has no attribute 'agent'

In [ ]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'table_info': '\nCREATE TABLE [Banking_Data] (\n\t[Property_ID] SMALLINT NOT NULL, \n\t[Property_Type] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, \n\t[Property_Address] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, \n\t[Property_Size] SMALLINT NOT NULL, \n\t[Square_Footage] SMALLINT NOT NULL, \n\t[Bedrooms] TINYINT NOT NULL, \n\t[Bathrooms] TINYINT NOT NULL, \n\t[Year_Built] SMALLINT NOT NULL, \n\t[Purchase_Price] INTEGER NOT NULL, \n\t[Sale_Price] INTEGER NOT NULL, \n\t[Rental_Income] SMALLINT NOT NULL, \n\t[Operating_Expenses] SMALLINT N

In [ ]:
from langchain.memory import ChatMessageHistory
#memory = ChatMessageHistory(session_id="test-session")

In [ ]:
agent.run("Mean Final Prices of New York Location with 3 Bedrooms")

AttributeError: 'RunnableSequence' object has no attribute 'run'

In [ ]:
"""from langchain_core.runnables.history import RunnableWithMessageHistory
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)"""

'from langchain_core.runnables.history import RunnableWithMessageHistory\nagent_with_chat_history = RunnableWithMessageHistory(\n    agent_executor,\n    # This is needed because in most real world scenarios, a session id is needed\n    # It isn\'t really used here because we are using a simple in memory ChatMessageHistory\n    lambda session_id: memory,\n    input_messages_key="input",\n    history_messages_key="chat_history",\n)'

In [ ]:
"""agent_with_chat_history.invoke(
    {"input": "Total Rental Income and Operating Expenses by Location use chat history to answer"},
    config={"configurable": {"session_id": "<foo>"}},
)"""

'agent_with_chat_history.invoke(\n    {"input": "Total Rental Income and Operating Expenses by Location use chat history to answer"},\n    config={"configurable": {"session_id": "<foo>"}},\n)'

In [ ]:

#Assessing Buyer Financial Health
#I WANT THE HIGHEST MONTHLY SALARY?
#Calculates the total monthly salary and total current debts for each buyer.
#Total Rental Income and Operating Expenses by Location
#Properties with High Return on Investment